In [1]:
import pdal
import numpy as np

In [2]:
%load_ext autotime

In [3]:
pipeline="""{
  "pipeline": [
    {
        "type": "readers.las",
        "filename": "arquivos/MDS/MDS_3314-231.laz"
    },
    {
        "type": "filters.sample",
        "radius": 1
    }
  ]
}"""

In [4]:
r = pdal.Pipeline(pipeline)
r.validate()
r.execute()
pts = r.arrays[0]

In [5]:
pts

array([(333499.29, 7395564.35, 735.34, 18, 1, 1, 0, 0, 6, -14., 17, 6, 356569.36606547),
       (333499.23, 7395565.79, 735.41, 18, 1, 1, 0, 0, 6, -14., 17, 6, 356569.36609045),
       (333499.2 , 7395566.79, 735.25, 15, 1, 1, 0, 0, 6, -14., 17, 6, 356569.36610547),
       ...,
       (333520.96, 7395571.  , 732.5 ,  8, 1, 1, 1, 0, 6,  13., 40, 6, 363044.34273991),
       (333521.09, 7395571.84, 731.83, 18, 1, 1, 1, 0, 6,  13., 40, 6, 363044.3609089 ),
       (333520.95, 7395571.91, 729.86, 18, 1, 1, 0, 0, 6,  13., 40, 6, 363044.36412489)],
      dtype=[('X', '<f8'), ('Y', '<f8'), ('Z', '<f8'), ('Intensity', '<u2'), ('ReturnNumber', 'u1'), ('NumberOfReturns', 'u1'), ('ScanDirectionFlag', 'u1'), ('EdgeOfFlightLine', 'u1'), ('Classification', 'u1'), ('ScanAngleRank', '<f4'), ('UserData', 'u1'), ('PointSourceId', '<u2'), ('GpsTime', '<f8')])

In [6]:
pt_edif = pts['Classification'] == 6
len(pts[pt_edif])

300792

In [7]:
pt_ground = pts['Classification'] == 2
len(pts[pt_ground])

15890

In [8]:
len(pts[pt_ground | pt_edif])

316682

In [9]:
# Escolhendo um ponto aleatório no Ground

pt = np.random.choice(pts[pt_ground])

In [10]:
coord = ['X', 'Y', 'Z']

In [11]:
pt[coord]

(333064.64, 7395475.68, 729.97)

In [12]:
# Centralizando a visão em 1.5 metros acima do solo do ponto escolhido

pts_0 = np.array([pts[pt_ground | pt_edif]['X'] - pt['X'],
         pts[pt_ground | pt_edif]['Y'] - pt['Y'],
         pts[pt_ground | pt_edif]['Z'] - (pt['Z'] + 1.5)]).T

In [28]:
## TODO
# Reprojetar os pontos para a circunferência da terra

In [29]:
## Filtrando pontos acima da visão horizontal

pts_0 = pts_0[pts_0[:, 2] >= 0.0]

In [31]:
len(pts_0)

296297

In [30]:
# Calcluando Azimute, ou melhor, um angulo começando Leste 
# crescendo no sentido anti-horario e negativo para todo Y < 0

teste_az = [[0., 1],
           [1., 0.],
           [0., -1.],
           [-1., 0.]]

teste_az = np.array(teste_az)

az = np.arctan2(teste_az[:, 1], teste_az[:, 0]) * 180 / np.pi
az

array([ 90.,   0., -90., 180.])

In [15]:
# Calculando o angulo de orientação de cada ponto

orient = np.arctan2(pts_0[:, 1], pts_0[:, 0]) * 180 / np.pi
# orient

In [16]:
# Calculando o angulo de altura de cada ponto

# np.arctan2(10, 1000) * 180 / np.pi
# np.sqrt(np.square(4) + np.square(3))

inclin = np.arctan2(pts_0[:, 2], np.sqrt(np.square(pts_0[:, 0]) + np.square(pts_0[:, 1]))) * 180 / np.pi

In [40]:
# Calculando a distância até o ponto

np.sqrt(np.square(1) + np.square(1) + np.square(10))

10.099504938362077

In [33]:
# sorted(inclin, reverse=True)

In [34]:
orient

array([11.53030756, 11.71399842, 11.84113908, ..., 11.08668748,
       11.79877106, 11.89650228])

In [35]:
# Separando em fases

qt_fases = 60
fases = np.arange(-180., 180., 360./qt_fases)

pt_bins = np.digitize(orient, fases)

In [36]:
np.histogram(orient, fases)

(array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,  3119, 16676, 20158, 15686,
        23031, 17544, 20379, 18257, 18341, 13456, 15111, 13979, 11506,
        12495, 14029, 12125, 15287, 13432,  8327,  3562,  1826,   839,
          505,   471,   979,  1203,  1004,   728,   512,   459,   430,
          525,   260,    56,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0]),
 array([-180., -174., -168., -162., -156., -150., -144., -138., -132.,
        -126., -120., -114., -108., -102.,  -96.,  -90.,  -84.,  -78.,
         -72.,  -66.,  -60.,  -54.,  -48.,  -42.,  -36.,  -30.,  -24.,
         -18.,  -12.,   -6.,    0.,    6.,   12.,   18.,   24.,   30.,
          36.,   42.,   48.,   54.,   60.,   66.,   72.,   78.,   84.,
          90.,   96.,  102.,  108.,  114.,  120.,  126.,  132.,  138.,
         144.,  150.,  156.,  162.,  168.,  174.]))

In [21]:
np.unique(pt_bins)

array([15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31,
       32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48])

In [22]:
np.arange(1, qt_fases + 1)

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60])

In [23]:
list(map(lambda x: [x, np.nanmax(inclin[pt_bins == x])] if len(inclin[pt_bins == x]) > 0 else [x, 0.], np.arange(1, qt_fases + 1)))
# list(map(lambda x: len(orient[pt_bins == x]), fases))
# np.nanmax(inclin[pt_bins == 3]) if len(inclin[pt_bins == 3]) > 0

[[1, 0.0],
 [2, 0.0],
 [3, 0.0],
 [4, 0.0],
 [5, 0.0],
 [6, 0.0],
 [7, 0.0],
 [8, 0.0],
 [9, 0.0],
 [10, 0.0],
 [11, 0.0],
 [12, 0.0],
 [13, 0.0],
 [14, 0.0],
 [15, 17.85252259855616],
 [16, 25.242791922406763],
 [17, 28.310030340545957],
 [18, 27.473663021422563],
 [19, 24.616393958775653],
 [20, 17.97870653728566],
 [21, 42.27715651756928],
 [22, 43.48967957835123],
 [23, 43.44146757212334],
 [24, 24.543818073798022],
 [25, 21.359016523485682],
 [26, 17.37704264015881],
 [27, 15.00709332322732],
 [28, 15.34345248514329],
 [29, 17.515664772161053],
 [30, 21.147536723316357],
 [31, 20.97885858094463],
 [32, 20.210493108236516],
 [33, 12.779385339468524],
 [34, 78.73296507375849],
 [35, 79.65176865762436],
 [36, 80.45533363934106],
 [37, 80.76959476052899],
 [38, 81.89933452746641],
 [39, 81.45847743058506],
 [40, 81.75303286892346],
 [41, 81.82725992851574],
 [42, 81.74255440808165],
 [43, 81.68605377333371],
 [44, 81.60827912822228],
 [45, 81.38967576208],
 [46, 80.98670161655853],
 [

In [49]:
# A relação entre a densidade de pontos e a distância para obstrução do céu visível
# Quanto mais próximo um ponto está do observador, maior a obstrução que ele deve promover
# Os parâmetros portanto são distância ao observador e a distância máxima entre pontos

# Os bins então vão responder não mais a um angulo, mas uma faixa de angulos

# o Angulo é dado por arctan2(dist_max_entre_pontos/2, distancia_do_observador)

# densidade em raio
raio = 1

desvio_graus = np.degrees(np.arctan2(raio/2, \
                      np.sqrt(np.square(pts_0[:, 0]) + \
                              np.square(pts_0[:, 1]) + \
                              np.square(pts_0[:, 2]))) * 2)

desvio_graus

# Depois refatorar a função de máximo ângulo da fase

array([0.12915527, 0.12908778, 0.12903739, ..., 0.11497785, 0.12290731,
       0.12282861])

In [91]:
# Refatorando o calculo das fases

# list(map(lambda x: \
#          [x, np.nanmax(inclin[pt_bins == x])] if len(inclin[pt_bins == x]) > 0 else [x, 0.], np.arange(1, qt_fases + 1)))

orient + desvio_graus
list(map(lambda x: x, fases))

len(pts_0[((orient + desvio_graus >= 6.) | (orient - desvio_graus >= 6.) ) & ((orient + desvio_graus < 6. + 6) | (orient - desvio_graus < 6. + 6) ) ])

14278

In [87]:
np.array([True, True]) | np.array([False, True])

array([ True,  True])

In [25]:
# Calcular o percentual de cada fuso
# Sendo que a área do fuso é a área da esfera 4 x Pi x r x r dividido pela quantidade de fusos
# em cada fuso subtrair a quantidade de céu visível, dao pela fórmula da calota da esfera
# 2 x Pi x r x h (sendo necessário calcular h e considenrando r constante)

In [ ]:
## TODO
# Pensar em como iterar sobre as folhas!

array([(333499.29, 7395564.35, 735.34, 18, 1, 1, 0, 0, 6, -14., 17, 6, 356569.36606547),
       (333499.23, 7395565.79, 735.41, 18, 1, 1, 0, 0, 6, -14., 17, 6, 356569.36609045),
       (333499.2 , 7395566.79, 735.25, 15, 1, 1, 0, 0, 6, -14., 17, 6, 356569.36610547),
       ...,
       (333520.96, 7395571.  , 732.5 ,  8, 1, 1, 1, 0, 6,  13., 40, 6, 363044.34273991),
       (333521.09, 7395571.84, 731.83, 18, 1, 1, 1, 0, 6,  13., 40, 6, 363044.3609089 ),
       (333520.95, 7395571.91, 729.86, 18, 1, 1, 0, 0, 6,  13., 40, 6, 363044.36412489)],
      dtype=[('X', '<f8'), ('Y', '<f8'), ('Z', '<f8'), ('Intensity', '<u2'), ('ReturnNumber', 'u1'), ('NumberOfReturns', 'u1'), ('ScanDirectionFlag', 'u1'), ('EdgeOfFlightLine', 'u1'), ('Classification', 'u1'), ('ScanAngleRank', '<f4'), ('UserData', 'u1'), ('PointSourceId', '<u2'), ('GpsTime', '<f8')])